In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\jarod\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
df_jobs = pd.read_csv('../../data/raw_data/datasets/job_raw_V0')

In [4]:
df_jobs.head()

,id,job
0,0,Desarrollador de Software Junior - Aplicacione...
1,1,Ingeniero de Software Full Stack - Tecnologías...
2,2,Desarrollador de Software Frontend - Experienc...
3,3,Desarrollador de Software Backend - Arquitectu...
4,4,Ingeniero de Software de Juegos - Desarrollo d...


**Eliminar caracter especial salto de linea '\n'**

In [5]:
def delete_line_break(df):
    df_copy = df.copy()
    df_copy.iloc[:, 1] = df_copy.iloc[:, 1].apply(lambda x: x.replace('\n', ' '))
    return df_copy

In [6]:
df_jobs=delete_line_break(df_jobs)

In [9]:
df_jobs.head()

,id,job
0,0,Desarrollador de Software Junior - Aplicacione...
1,1,Ingeniero de Software Full Stack - Tecnologías...
2,2,Desarrollador de Software Frontend - Experienc...
3,3,Desarrollador de Software Backend - Arquitectu...
4,4,Ingeniero de Software de Juegos - Desarrollo d...


**Modelado**

In [10]:
# Carga del modelo y tokenizador
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [11]:
# Tokenizamos los textos
tokenized_inputs = tokenizer(df_jobs['job'].to_list(), padding=True, 
                             truncation=True, max_length=768, return_tensors="tf")

In [12]:
# Extraemos el resultado en funcion de las representaciones de MiniLM
outputs = model(tokenized_inputs)

In [13]:
# Capturamos los embeddings
embeddings = outputs.last_hidden_state

In [14]:
# Normaliza los vectores de embedding para asegurarte de que tengan longitud unitaria
normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [15]:
def calculate_cosine(mean_embedding_1,mean_embedding_2):
    similarity_score = cosine_similarity([mean_embedding_1], [mean_embedding_2])[0, 0]
    return similarity_score

In [17]:
def similitud_job(job_1, job_2):
    
    # Calcular la media de los embeddings
    mean_embedding_1 = np.mean(normalized_embeddings[job_1], axis=0)
    mean_embedding_2 = np.mean(normalized_embeddings[job_2], axis=0)

    # Calcular la similitud coseno entre los embeddings
    similarity_score = calculate_cosine(mean_embedding_1,mean_embedding_2)

    # Mostrar en pantalla resultados
    print(f"La similaridad entre la oferta {job_1} y {job_2} es de:", similarity_score)
    print('\n------------')
    print(f'Oferta con id {job_1}')
    print(df_jobs['job'][job_1])
    print('\n------------')
    print(f'Oferta con id {job_2}')
    print(df_jobs['job'][job_2])

In [22]:
similitud_job(10, 23)

La similaridad entre la oferta 10 y 23 es de: 0.6735363

------------
Oferta con id 10
Ingeniero de Redes Senior Acerca del Empleo: Estamos buscando un Ingeniero de Redes Senior para unirse a nuestro equipo dinámico. En este papel, serás responsable de diseñar, implementar y mantener nuestra infraestructura de red para garantizar la conectividad y la seguridad de nuestros sistemas.  Responsabilidades:  Diseñar y configurar redes LAN/WAN para garantizar un rendimiento óptimo. Implementar medidas de seguridad de red, como firewalls y sistemas de detección de intrusiones. Colaborar con otros equipos para resolver problemas de red y optimizar el rendimiento. Realizar pruebas de rendimiento y diagnóstico para identificar y resolver problemas de red. Mantener la documentación técnica actualizada sobre la infraestructura de red. Cualificaciones:  Licenciatura en Ingeniería Informática, Telecomunicaciones o campo relacionado. Experiencia previa en la configuración y administración de dispositi